# Plotting with *PyMovements*

*PyMovements* provides a collection of built-in plotting functions 
to visualize gaze data in both temporal and spatial dimensions.

These functions make it easy to explore and present your data, 
from individual trial visualizations to aggregated participant-level analyses.

In this tutorial, you’ll learn how to:

- load and prepare a sample dataset for plotting
- compute the necessary properties
- Plot gaze traces over time (traceplot)
- Visualize fixations and saccades on a stimulus (scanpathplot)
- Generate heatmaps showing gaze density (heatmap)
- Plot the saccadic main sequence (main-sequence)
- style the plot

All examples use the small and reproducible :doc:`ToyDataset <../../datasets/ToyDataset>` 
that comes with PyMovements.

## Loading and preprocessing data

In [ ]:
import pymovements as pm

dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

# convert the raw x and y coordinates in pixels to degrees in visual angle
dataset.pix2deg()

# convert these positions into velocitites
dataset.pos2vel()

Check if all the expected columns are in place

In [ ]:
#  You should see the following columns: time, stimuli_x, stimuli_y, text_id
#  page_id, pixel, position, velocity
dataset.gaze[0]

## Detecting events and computing properties

In [ ]:
# detect saccades
dataset.detect_events('microsaccades')
# compute amplitude and peak velocity of the detected saccades
dataset.compute_event_properties(['amplitude', 'peak_velocity'])

Verify that the desired columns are available

In [ ]:
#  the DataFrame with detected events should now contain the following columns:
#  text_id, page_id, name, onset, offset, duration, amplitude, peak_velocity
dataset.events[0]

## Plotting the Saccadic Main Sequence

Now we just pass the event dataframe to the plotting function:

In [ ]:
# only showing the first three event dataframes here.
# note that you can adjust the styling of the plot, e.g. setting a low
# alpha value allows you to see overlapping data points
for event_df in dataset.events[:3]:
    pm.plotting.main_sequence_plot(
        event_df,
        title='Main sequence plot for '
        f'text {event_df[0, "text_id"]}, '
        f'page {event_df[0, "page_id"]}',
        alpha=0.3,
        color='green',
        marker='x',
        marker_size=30,
    )

## Making the Traceplot

A traceplot displays the continuous horizontal and vertical gaze coordinates of one or more trials, allowing you to inspect temporal dynamics such as fixations, saccades, and blinks.

Traceplots are useful for:
- Verifying that gaze data have been parsed and aligned correctly.
- Exploring viewing behavior across conditions or participants.
- Identifying artifacts or data quality issues.

A basic traceplot can be created with only a `Gaze` object:

In [ ]:
pm.plotting.traceplot(dataset.gaze[0])

## Creating the Scanpath Plot

This scanpath plot visualizes fixations and saccades on a stimulus using
`pymovements.plotting.scanpathplot()`.

In [ ]:
pm.plotting.scanpathplot(dataset.gaze[0])

## Heatmap Plotting

The heatmap shows the distribution of gaze positions across the experiment screen, with color values indicating the time spent at each position in seconds.

Let's create a heatmap using the `heatmap` function from the `pymovements` library. We will use the default `gridsize` of 10x10 with interpolation and display the colorbar.

To better understand the effect of the `gridsize` parameter on the heatmap, we can turn off the interpolation. By doing this, we can clearly visualize the individual bins used to calculate the heatmap. With interpolation turned off, the heatmap will display the raw bin values rather than a smoothed representation.

In [ ]:
figure = pm.plotting.heatmap(
    gaze=dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation On',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10]
)

Increasing the `gridsize` parameter results in a finer grid and more detailed heatmap representation. With a higher grid size, we divide the plot into smaller bins, which can capture more nuances in the data distribution

In [ ]:
figure = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Higher Grid Size',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[25, 25]
)

In [ ]:
pm.plotting.heatmap(dataset.gaze[2])